In [1]:
import tensorflow as tf

In [1]:
graph_path = '/data/data/parviz/tmp/retinanet/retinanet_061518/640px_fp32/frozen_retinanet.pb'

In [2]:
from IPython.display import clear_output, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import tensorflow as tf
import numpy as np

In [5]:

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
    with open(graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        # Swap out iterator for Placeholder.
        image = tf.placeholder(tf.float32, shape=(1, 640, 640, 3))
        _ = tf.import_graph_def(output_graph_def, name="", input_map={'IteratorGetNext': image})
    
    
    with tf.gfile.GFile(graph_path.replace('.pb', '_cleaned.pb'), "wb") as f:
        f.write(tf.get_default_graph().as_graph_def().SerializeToString())
        
    show_graph(tf.get_default_graph().as_graph_def())
    

NotFoundError: /data/data/parviz/tmp/retinanet_cleaned/retinanet_cleaned_061518/640px_fp32/frozen_retinanet_cleaned.pb; No such file or directory

In [4]:

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
    with open(graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        # Swap out iterator for Placeholder.
        image = tf.placeholder(tf.float32, shape=(1, 640, 640, 3))
        _ = tf.import_graph_def(output_graph_def, name="", input_map={'IteratorGetNext': image})
    
    #[op for op in graph.get_operations()]
    for op in tf.get_default_graph().get_operations():
        if op.type not in ['Const', 'Identity', 'BiasAdd', 'FusedBatchNorm', 'Relu']:
            print('[' + op.type + '] ' + op.name)
            if op.type in ['Conv2D']:
                print(op)
                #print(op.__dict__)
                

[Placeholder] Placeholder
[OneShotIterator] OneShotIterator
[IteratorGetNext] IteratorGetNext
[Pad] resnet50/Pad
[Conv2D] resnet50/conv2d/Conv2D
name: "resnet50/conv2d/Conv2D"
op: "Conv2D"
input: "resnet50/Pad"
input: "resnet50/conv2d/kernel/read"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "data_format"
  value {
    s: "NHWC"
  }
}
attr {
  key: "dilations"
  value {
    list {
      i: 1
      i: 1
      i: 1
      i: 1
    }
  }
}
attr {
  key: "padding"
  value {
    s: "VALID"
  }
}
attr {
  key: "strides"
  value {
    list {
      i: 1
      i: 2
      i: 2
      i: 1
    }
  }
}
attr {
  key: "use_cudnn_on_gpu"
  value {
    b: true
  }
}

[MaxPool] resnet50/max_pooling2d/MaxPool
[Conv2D] resnet50/conv2d_1/Conv2D
name: "resnet50/conv2d_1/Conv2D"
op: "Conv2D"
input: "resnet50/initial_max_pool"
input: "resnet50/conv2d_1/kernel/read"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "data_format"
  value {
    s: "NHWC"
  }
}
attr {
  key: 

In [6]:

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
    with open(graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        # Swap out iterator for Placeholder.
        image = tf.placeholder(tf.float32, shape=(1, 640, 640, 3))
        _ = tf.import_graph_def(output_graph_def, name="", input_map={'IteratorGetNext': image})
    
    config = tf.ConfigProto(device_count = {'GPU': 0})
    with tf.Session(config=config) as sess:
        tensors = []
        tensors.append(tf.get_default_graph().get_tensor_by_name("resnet_fpn/l4/Conv2D:0"))
        tensors.append(tf.get_default_graph().get_tensor_by_name("resnet_fpn/l5/Conv2D:0"))
        tensors.append(tf.get_default_graph().get_tensor_by_name("resnet_fpn/l3/Conv2D:0"))
        
        tensors.append(tf.get_default_graph().get_tensor_by_name("resnet_fpn/nearest_upsampling/Reshape_1:0"))
        
        tensors.append(tf.get_default_graph().get_tensor_by_name("resnet_fpn/nearest_upsampling_1/Reshape_1:0"))
        
        
        outputs = sess.run(tensors, feed_dict={image: np.random.random(size=(1,640,640,3))})
        for output in outputs:
            print(output.shape)
        
    
    


(1, 40, 40, 256)
(1, 20, 20, 256)
(1, 80, 80, 256)
(1, 40, 40, 256)
(1, 80, 80, 256)


In [7]:
#!pip install graphviz
#!pip install tfgraphviz

In [10]:
import tfgraphviz as tfg

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
    with open(graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        # Swap out iterator for Placeholder.
        image = tf.placeholder(tf.float32, shape=(1, 640, 640, 3))
        _ = tf.import_graph_def(output_graph_def, name="", input_map={'IteratorGetNext': image})
    
    g = tfg.board(tf.get_default_graph(), depth=10)
    g.view()

filepath = G.gv.pdf
